In [1]:
!pip install -q underthesea
# !pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00


In [27]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AdamW, AutoModelForCausalLM
from torch.optim import Adam
import gdown
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from huggingface_hub import login
from google.colab import files
import shutil
import gdown
from typing import List
#from underthesea import sent_tokenize
from nltk import word_tokenize as lib_tokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#processing dataset

Dữ liệu

In [3]:
gdown.download('https://drive.google.com/uc?id=1Ebtb6ItULnfFqz0qYKdVkaror-GvuNJr')# training
gdown.download('https://drive.google.com/uc?id=1SA4i75wppxd2Qf6E3-8Qe2quIPlYMS9d')# dev

Downloading...
From: https://drive.google.com/uc?id=1Ebtb6ItULnfFqz0qYKdVkaror-GvuNJr
To: /content/vi_squad.csv
100%|██████████| 93.3M/93.3M [00:01<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SA4i75wppxd2Qf6E3-8Qe2quIPlYMS9d
To: /content/vi_squad_dev.csv
100%|██████████| 11.7M/11.7M [00:00<00:00, 51.2MB/s]


'vi_squad_dev.csv'

In [4]:
df = pd.read_csv('/content/vi_squad.csv')
# dev = pd.read_csv('/content/vi_squad_dev.csv')

In [5]:
df.head()

,context,question,answer
0,"Architecturally, trường học có một nhân vật Cô...",Ai đã làm Trinh nữ Mary bị cáo buộc xuất hiện ...,Saint Bernadette Soubirous
1,"Architecturally, trường học có một nhân vật Cô...",Cái gì ở trước tòa nhà chính của Notre Dame?,một bức tượng đồng của Chúa Kitô
2,"Architecturally, trường học có một nhân vật Cô...",Thánh đường của trái tim thiêng liêng tại Notr...,Tòa nhà chính
3,"Architecturally, trường học có một nhân vật Cô...",Glimpses ở Notre Dame là gì?,một nơi Marian của cầu nguyện và phản ánh
4,"Architecturally, trường học có một nhân vật Cô...",Cái gì ngồi trên đỉnh của tòa nhà chính tại No...,một bức tượng vàng của Trinh Mary


#Dataset/Dataloader

In [6]:
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, src_max_token_len=128, tgt_max_token_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.src_max_token_len = src_max_token_len
        self.tgt_max_token_len = tgt_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_row = self.data.iloc[idx]
        input_encoding = self.tokenizer(data_row['context'],
                                        truncation=True,
                                        max_length=self.src_max_token_len,
                                        padding='max_length',
                                        return_tensors='pt')

        output_encoding = self.tokenizer('{} Đáp án:\n {}'.format(data_row['question'], data_row['answer']),
                                          truncation=True,
                                          max_length=self.tgt_max_token_len,
                                          padding='max_length',
                                          return_tensors='pt')

        labels = output_encoding['input_ids']
        labels[labels == 0] = -100

        return dict(
            context=data_row['context'],
            question=data_row['question'],
            answer=data_row['answer'],
            input_ids=input_encoding['input_ids'].flatten(),
            attention_mask=input_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
        )


In [7]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [8]:
train_pairs, test_pairs = train_test_split(df, test_size=0.2, random_state=42)
train_pairs, dev_pairs = train_test_split(train_pairs, test_size=0.2, random_state=42)


In [9]:
train_dataset = TextDataset(train_pairs, tokenizer)
dev_dataset = TextDataset(dev_pairs, tokenizer)
test_dataset = TextDataset(test_pairs, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Train

In [10]:
epochs = 5
lr = 1e-4

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=lr)

best_dev_loss = float('inf')
for epoch in range(epochs):
    model.train()
    train_loop = tqdm(train_dataloader, leave=True)
    for batch in train_loop:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        train_loop.set_description(f'Epoch {epoch}')
        train_loop.set_postfix(loss=loss.item())

    model.eval()
    dev_loss = 0
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            dev_loss += outputs.loss.item()

    dev_loss /= len(dev_dataloader)

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        model.save_pretrained("my_vit5_finetuned")
        tokenizer.save_pretrained("my_vit5_finetuned")

model = AutoModelForSeq2SeqLM.from_pretrained("my_vit5_finetuned")
tokenizer = AutoTokenizer.from_pretrained("my_vit5_finetuned")
model.to(device)
model.eval()

test_loss = 0
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

test_loss /= len(test_dataloader)
print(f'Test loss: {test_loss}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   1%|▏         | 98/6786 [00:52<59:22,  1.88it/s, loss=2.14]


KeyboardInterrupt: 

Download before crash


In [12]:
model.save_pretrained("my_vit5_finetuned")
tokenizer.save_pretrained("my_vit5_finetuned")

shutil.make_archive('/content/my_vit5_finetuned', 'zip', 'my_vit5_finetuned')# zip

files.download('/content/my_vit5_finetuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
input_texts = """
BÀI 1  GIỚI THIỆU VỀ HỌC MÁY  Học xong bài này, em sẽ: • Giải thích được sơ lược về khái niệm Học máy
• Nêu được vai trò của Học máy trong những công việc như: lọc thư rác, chẩn đoán bệnh, phân tích thị trường, nhận dạng tiếng nói và chữ viết, dịch tự động,...
Cho Hình 1 và ba nhân phân loại là "vuông", "tròn", "tam giác". Cần gán nhãn phân loại cho từng đối t
ượng trong Hình 1. Em hãy trả lời các câu hỏi sau: 1) Nếu con người thực hiện thì nhiệm vụ trên là dễ hay khó? 2) Theo em, máy tính có thể tự động thực hiện nhiệm vụ trên thay cho con người hay không? Lập trình để máy tính làm công việc này là dễ hay khó? 1 Khái niệm Học máy Bài Giới thiệu về Trí tuệ nhân tạo ở Chủ đề A đã đề cập đến Học máy như một nhánh nghiên cứu trong ngành AI nhằm làm cho máy tính có khả năng học từ dữ liệu. Bài học này sẽ giúp các em hiểu sâu hơn về Học máy
Học máy huấn luyện máy tính để nó có thể tự động phát hiện ra các mối quan hệ có trong dữ liệu. Học máy có thể giải quyết nhiều loại bài toán khác nhau, tiêu biểu là bài toán phân loại và bài toán phân cụm
Để huấn luyện máy tính học từ dữ liệu, người ta dùng một tập dữ liệu gọi là dữ liệu huấn luyện. Máy tính thực hiện thuật toán trên tập dữ liệu huấn luyện để có khả năng tự thực hiện những nhiệm vụ tương tự trên tập dữ liệu mới. Dữ liệu huấn luyện mô tả các đối tượng trong thế giới thực. Mỗi đối tượng tương ứng với một mẫu dữ liệu
a) Mô hình học máy Thuật toán học máy: Từ tập dữ liệu đầu vào, thuật toán học máy rút ra các thông tin liên quan tới dữ liệu, các đặc điểm chung quan trọng.... Từ đó giúp máy tính học cách phân biệt giữa các mẫu dữ liệu khác nhau hoặc nhóm các mẫu dữ liệu thành các cụm nhiều mẫu tương tự nhau
Mô hình học máy: Các mô hình học máy được tạo ra từ các thuật toán học máy và trải qua quá trình huấn luyện bằng cách sử dụng dữ liệu huấn luyện (Hình 2). Thực hiện thuật toán học máy trên tập dữ liệu huấn luyện tức là huấn luyện mô hình học máy
Hình 2. Minh hoạ mô hình học máy  Thuật toán học máy  Dữ liệu huấn luyện  Mô hình học máy  Mô hình  Áp dụng mô hình  Dữ liệu kiểm tra Quá trình huấn luyện nhằm tạo ra mô hình học máy để giải quyết một bài toán cụ thể. Áp dụng mô hình cho phần dữ liệu chưa dùng trong huấn luyện để đánh giá mô hình. Mô hình được đưa vào sử dụng thực tế nếu kết quả đánh giá đáp ứng yêu cầu ứng dụng
b) Quy trình học máy Tuỳ theo lĩnh vực ứng dụng và bài toán cụ thể, người ta chọn tập dữ liệu, dùng thuật toán học máy và cách đánh giá kết quả huấn luyện thích hợp
Quy trình học máy có thể mô tả như sau: 1. Thu thập dữ liệu: Chọn dữ liệu phù hợp với bài toán cụ thể. Dữ liệu có thể được chọn từ nhiều nguồn, có khuôn dạng khác nhau, có thể là dữ liệu có cấu trúc ví dụ như các bản ghi trong cơ sở dữ liệu hoặc phi cấu trúc. Tập dữ liệu thu thập được là dữ liệu thô, chưa sẵn sàng để sử dụng trong quá trình huấn luyện
2. Chuẩn bị dữ liệu: Làm sạch, loại bỏ nhiễu, bổ sung các giá trị thiếu và chuyển đổi dữ liệu sang một khuôn dạng phù hợp. Chia dữ liệu đã chuẩn bị thành hai phần
Một phần lớn dữ liệu được dùng làm dữ liệu huấn luyện và phần còn lại dùng làm dữ liệu để đánh giá mô hình
3. Xây dựng mô hình: Chọn thuật toán học máy phù hợp với bài toán và dữ liệu đã chuẩn bị. Có nhiều loại thuật toán học máy như: hồi quy tuyến tính, cây quyết định, mạng nơ ron,..
– Huấn luyện mô hình để mô hình học từ dữ liệu và trở nên thích ứng với bài toán cụ thể đó
4. Đánh giá mô hình: Áp dụng mô hình sau huấn luyện cho phần dữ liệu dành để đánh giá mô hình. Đối chiếu kết quả với tiêu chí đánh giá để xác định mức độ đáp ứng yêu cầu ứng dụng. Việc huấn luyện và đánh giá thường được thực hiện nhiều lần cho tới khi mô hình đạt yêu cầu mong muốn. Nếu kết quả đánh giá chưa đạt, cần tiếp tục cải thiện mô hình. Để cải thiện mô hình, có thể: phân chia lại dữ liệu huấn luyện và dữ liệu dành để đánh giá, bổ sung thêm dữ liệu mới, điều chỉnh các tham số của thuật toán học máy hoặc sử dụng thuật toán học máy khác
 5. Triển khai ứng dụng mô hình: Sử dụng mô hình đã được huấn luyện thành công vào ứng dụng Học máy trong bài toán thực tế
 Có thể chia Học máy thành hai loại chính: học có giám sát và học không giám sát
 2 Học có giám sát Trong học có giám sát, tập dữ liệu huấn luyện gồm các mẫu dữ liệu được liên kết với đầu ra tương ứng, gọi là nhãn. Máy tính học để phát hiện ra mối quan hệ giữa các mẫu dữ liệu với nhân. Sau khi học xong, máy tính có thể đưa ra dự đoán nhãn cho dữ liệu mới
Học có giám sát có thể dùng để giải quyết nhiều loại bài toán khác nhau, trong đó có bài toán phân loại
a) Bài toán phân loại Xét các bài toán sau: 1) Hệ thống thư điện tử cần xác định các email nghi là thư rác và đánh dấu nó thuộc loại “spam” để cảnh báo cho người nhận. Những email còn lại thuộc loại "no spam". 2) Ngân hàng cần xác định khách hàng thuộc loại "tốt", "bình thường" hay "xấu" để quyết định hạn mức cho vay và lãi suất áp dụng
Theo em, hai bài toán trên và nhiệm vụ ở phần khởi động có những điểm chung là gì? Có một số nhãn phân loại cho trước. Việc gán cho mỗi đối tượng một nhãn phân loại tuỳ theo các thuộc tính đặc trưng của nó là bài toán phân loại. Hai bài toán nêu trong Hoạt động 1 là bài toán phân loại. Bài toán 1 có hai nhãn phân loại là “spam” và "nonspam"; bài toán 2 có ba nhãn phân loại là “tốt”, “bình thường” và “xấu”. Học máy giúp xây dựng mô hình phân loại để phân loại thư rác, phân loại khách hàng vay tín dụng. Hình 3 minh hoạ vai trò của Học máy trong phân loại thư rác. Biểu tượng email màu đỏ thể hiện thư có các thuộc tính đặc trưng của thư rác; biểu tượng email màu xám thể hiện thư không có những thuộc tính đặc trưng của thư rác. Mô hình phân loại thư rác đã được huấn luyện thành công bằng học có giám sát gán nhãn “spam” hay "no spam" cho thư mới nhận được
b) Dữ liệu huấn luyện Mỗi đối tượng cần phân loại được mô tả bởi một số thông tin là các thuộc tính đặc trưng của nó. Ví dụ, việc phân loại email là “spam” hay “no spam” dựa vào một số thông tin như: địa chỉ người gửi, địa chỉ người nhận, dòng tiêu đề, sự có mặt của những từ đặc trưng cho thư rác...
Dữ liệu huấn luyện là các mẫu dữ liệu về một số email đã biết trước là thư rác và một số email khác không là thư rác
c) Huấn luyện và đánh giá mô hình Máy tính được huấn luyện để sử dụng dữ liệu huấn luyện và tự dự đoán nhãn phân loại theo thuật toán học máy. Nhân phân loại được xác định khi biết giá trị các thuộc tính đặc trưng của đối tượng. Áp dụng mô hình cho phần dữ liệu dùng để đánh giá sẽ nhận được dữ liệu kiểm tra. So sánh nhãn đã biết với nhãn do mô hình dự đoán để đánh giá mô hình. Mục tiêu huấn luyện nhằm giảm thiểu nhân bị gán sai đến mức ngưỡng chấp nhận được
3)Học không giám sát Khác với học có giám sát, học không giám sát được thực hiện với tập dữ liệu không có nhãn. Học không giám sát có thể dùng để huấn luyện máy tính giải quyết nhiều loại bài toán khác nhau, trong đó có bài toán phân cụm
a) Bài toán phân cụm Cho một số quả, theo em máy tính có thể trả lời được có mấy loại quả hay không? Phân cụm là chia tập đối tượng thành các cụm dựa trên sự tương tự và khác biệt của các đối tượng. Các đối tượng trong cùng một cụm có các đặc điểm tương tự nhau. Các đối tượng trong các cụm khác nhau thì có đặc điểm khác nhau. Học không giám sát giúp xây dựng mô hình phân cụm. Hình 4 minh họa vai trò của Học máy trong phân cụm các quả. Mô hình phân cụm các quả đã được huấn luyện thành công bằng học không giám sát gồm các quả thành ba cụm dựa vào đặc điểm các loại quả
 ĐẦU RA  Hình 4. Ứng dụng Học máy phân cụm các quả  b) Dữ liệu huấn luyện  Tập dữ liệu huấn luyện gồm các mẫu dữ liệu mô tả các thuộc tính đặc trưng của đối tượng. Ví dụ, trường hợp phân cụm các quả, các thuộc tính đặc trưng của mỗi quả là hình dạng, kích thước, màu sắc.... Trong dữ liệu không có tên các loại quả, tức là không có nhãn kèm theo
 c) Huấn luyện và đánh giá mô hình  Dựa trên thông tin về sự tương tự của các mẫu dữ liệu, thuật toán học máy sẽ nhóm các mẫu dữ liệu thành các cụm. Có một số cách khác nhau để phân cụm như: dựa trên mật độ các mẫu dữ liệu trong một vùng, dựa trên phân phối xác suất của các mẫu dữ liệu. Mức độ tương tự giữa hai mẫu dữ liệu có thể biểu diễn trực quan bằng “khoảng cách" giữa hai mẫu. Thuật toán học máy có thể phân cụm dựa trên khoảng cách giữa các mẫu dữ liệu và khoảng cách từ mẫu dữ liệu đến tâm cụm
 Khác với học có giám sát, kết quả phân cụm được đánh giá trực tiếp dựa vào tính chất của dữ liệu và yêu cầu phân tích dữ liệu, vì không có nhãn để kiểm tra là đúng hay sai
 4) Một số ứng dụng của Học máy  Học máy được ứng dụng để lọc thư rác, phân loại khách hàng vay tín dụng, phân cụm các quả như đã trình bày ở trên. Sau đây là một số ứng dụng khác của Học máy
 a) Nhận dạng tiếng nói  Máy tính có thể nhận dạng tiếng nói nhờ Học máy. Dữ liệu đầu vào là các đoạn tiếng nói và nhờ các thuật toán học máy sẽ xây dựng được mô hình nhận dạng tiếng nói
Nhận dạng tiếng nói được ứng dụng trong: chuyển lời nói thành văn bản; tìm kiếm bằng lời nói, điều khiển thiết bị thông minh bằng lời nói, dịch vụ trả lời tự động, chatbot trợ lí ảo hay xác thực bằng sinh trắc học tiếng nói,... Hình 5 minh hoạ nhận dạng tiếng nói sau đó chuyển thành văn bản trên ứng dụng Google Dịch
 b) Nhận dạng chữ viết  Máy tính có thể nhận dạng chữ viết nhờ Học máy. Dữ liệu đầu vào là các kí tự hoặc các câu đã được số hoá và nhờ các thuật toán học máy sẽ xây dựng được mô hình nhận dạng chữ viết
 Hình 6 minh hoạ một người đang viết ghi chú bằng chữ viết tay trên điện thoại thông minh. Những ghi chú bằng chữ viết tay này sẽ được chuyển thành văn bản và lưu thành tệp văn bản
 Nhận dạng chữ viết tay có thể chia thành hai chế độ, “tĩnh” và “động”. Ở chế độ tĩnh, cũng gọi là ngoại tuyển (offline), hình ảnh chữ viết tay được camera thu nhận và sau đó máy tính phân tích hình dạng chữ viết tay. Ở chế độ động, cũng gọi là trực tuyến (online), người trực tiếp viết chữ lên tấm cảm ứng, máy tính sẽ thu nhận chữ viết cùng lúc với thao tác viết và phân tích hình dạng chữ viết kết hợp với chuyển động, áp lực,... Phân tích chữ viết tay trực tuyến có thể ứng dụng để xác thực sinh trắc học chữ ký c) Dịch máy Dịch máy sử dụng Học máy để phân tích văn bản và dự đoán khả năng một từ  hoặc cụm từ cụ thể trong ngôn ngữ nguồn sẽ là từ hoặc cụm từ tương ứng nào trong  ngôn ngữ đích
 Google Dịch là một ví dụ tiêu biểu của ứng dụng dịch máy. Kết hợp nhận dạng  chữ viết tay, nhận dạng tiếng nói với dịch máy cung cấp nhiều tính năng và ứng  dụng đa dạng như: trợ lí ảo Google Assistant có chế độ phiên dịch cho phép trò chuyện  với người đối thoại nói bằng nhiều ngôn ngữ khác nhau; phiên dịch văn bản trực tiếp  bằng cách hướng camera vào văn bản, người dùng có thể xem kết quả dịch ngay  trên màn hình; Google Dịch có thể dịch từng từ, từng câu hay toàn bộ một trang web;  người dùng Gmail cũng có thể dễ dàng dịch email sang ngôn ngữ mong muốn
d) Chẩn đoán bệnh  Máy tính có thể chẩn đoán bệnh nhờ Học máy. Dữ liệu để chẩn đoán bệnh là các triệu chứng hoặc kết quả xét nghiệm y tế. Các nhãn phân loại là tên bệnh. Sử dụng Học máy để phân tích dữ liệu có thể dự đoán tên bệnh giúp các bác sĩ chẩn đoán nhanh hơn, tốt hơn
 e) Phân tích thị trường  Học máy không giám sát giúp xây dựng mô hình phân cụm dữ liệu khách hàng của doanh nghiệp. Dữ liệu khách hàng được phân cụm theo sự tương tự về giới tính, độ tuổi, nghề nghiệp hay về nhu cầu tiêu dùng, sở thích,... Kết quả phân cụm là các nhóm khách hàng mục tiêu thích hợp cho từng loại sản phẩm, dịch vụ. Từ đó, doanh nghiệp rút ra thông tin hữu ích để xây dựng chiến lược tiếp thị, giúp tăng doanh số, tăng thị phần, nâng cao hiệu quả hoạt động sản xuất kinh doanh
 - Mỗi phát biểu sau về Học máy là đúng hay sai?  a) Học không giám sát sử dụng dữ liệu huấn luyện không có nhãn
 b) Học có giám sát sử dụng dữ liệu kiểm tra để đánh giá kết quả huấn luyện
 c) Học có giám sát dành cho huấn luyện máy tính phân cụm
 d) Học có giám sát và không giám sát đều giúp máy tính giải quyết cùng một bài toán như nhau
 ChatGPT là một hệ thống AI nổi tiếng có nhiều khả năng khác nhau. Hãy kể ra một vài khả năng mà theo em có sự đóng góp của Học máy để phát triển hệ thống này
 Câu 1. Học máy là gì? Sự khác nhau giữa học có giám sát và học không giảm sát là gì?  Câu 2. Hãy kể một vài ứng dụng cụ thể trong đó có thể sử dụng Học máy để thực hiện nhiệm vụ phân loại và phân cụm
 Tóm tắt bài học  • Từ dữ liệu huấn luyện và thuật toán học máy xây dựng được mô hình học máy và  huấn luyện mô hình. Có hai loại mô hình học máy chính: học có giám sát và  học không giám sát tương ứng với dữ liệu huấn luyện là có gán nhân và không  có gán nhân
 • Các mô hình học máy giúp phân loại hoặc phân cụm các mẫu dữ liệu và được ứng dụng trong: lọc thư rác, chẩn đoán bệnh, nhận dạng tiếng nói và chữ viết, dịch tự động, phân tích thị trường
"""

In [28]:
#improve later
def split_context_according_to_desired_count(context: str, desired_count: int) -> List[str]:
    sents = sent_tokenize(context)
    sent_ratio = len(sents) / desired_count

    context_splits = []

    if sent_ratio < 1:
        return sents
    else:
        take_sents_count = int(sent_ratio + 1)

        start_sent_index = 0

        while start_sent_index < len(sents):
            context_split = ' '.join(sents[start_sent_index: start_sent_index + take_sents_count])
            context_splits.append(context_split)
            start_sent_index += take_sents_count - 1

    return context_splits

In [22]:
model = AutoModelForSeq2SeqLM.from_pretrained("my_vit5_finetuned")
tokenizer = AutoTokenizer.from_pretrained("my_vit5_finetuned")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

def generate_text(context, max_length=128):
    inputs = tokenizer.encode_plus(context, return_tensors="pt", max_length=128, truncation=True).to(device)

    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=200,
        top_p=0.9,
        top_k=50,
        temperature=0.7,
        repetition_penalty=1.2,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(context+'\n')
    print(generated_text)


In [31]:
context_split = split_context_according_to_desired_count(input_texts, 10)
for i in range(len(context_split)):
    generate_text(context_split[i])
    print()



BÀI 1  GIỚI THIỆU VỀ HỌC MÁY  Học xong bài này, em sẽ: • Giải thích được sơ lược về khái niệm Học máy
• Nêu được vai trò của Học máy trong những công việc như: lọc thư rác, chẩn đoán bệnh, phân tích thị trường, nhận dạng tiếng nói và chữ viết, dịch tự động,...
Cho Hình 1 và ba nhân phân loại là "vuông", "tròn", "tam giác". Cần gán nhãn phân loại cho từng đối tượng trong Hình 1. Em hãy trả lời các câu hỏi sau: 1) Nếu con người thực hiện thì nhiệm vụ trên là dễ hay khó? 2) Theo em, máy tính có thể tự động thực hiện nhiệm vụ trên thay cho con người hay không? Lập trình để máy tính làm công việc này là dễ hay khó? 1 Khái niệm Học máy Bài Giới thiệu về Trí tuệ nhân tạo ở Chủ đề A đã đề cập đến Học máy như một nhánh nghiên cứu trong ngành AI nhằm làm cho máy tính có khả năng học từ dữ liệu. Bài học này sẽ giúp các em hiểu sâu hơn về Học máy
Học máy huấn luyện máy tính để nó có thể tự động phát hiện ra các mối quan hệ có trong dữ liệu. Học máy có thể giải quyết nhiều loại bài toán khác nhau,